#### Pyod Functions

We are going to use some of the fuctions implemented in the library Pyod and sklearn:

#### Pyod

-  [Documentation](https://pyod.readthedocs.io/en/latest/)
-  [Github](https://github.com/yzhao062/pyod)


#### sk-learn
-  [Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html)


In [1]:
import pandas as pd
import numpy as np
import os 
import random
from Funciones import *
from scipy.spatial.distance import cdist
medida = 'euclidean'

random.seed(42)
import pyod

In [3]:
# Paths and Embeddings for the training set
set = 'Mobio'
a,b = load_train(set)

Img_Con_train_paths = reshape_paths(a[0][0])
Aud_Con_train_paths = reshape_paths(a[0][1])
Mix_Con_train_paths = reshape_paths(a[0][2])

Img_Descon_train_paths = reshape_paths(a[1][0])
Aud_Descon_train_paths = reshape_paths(a[1][1])
Mix_Descon_train_paths = reshape_paths(a[1][2])


Img_Con_train_embeddings = reshape_embed(b[0][0])
Aud_Con_train_embeddings = reshape_embed(b[0][1])
Mix_Con_train_embeddings = reshape_embed(b[0][2])

Img_Descon_train_embeddings = reshape_embed(b[1][0])
Aud_Descon_train_embeddings = reshape_embed(b[1][1])
Mix_Descon_train_embeddings = reshape_embed(b[1][2])


# Paths and Embeddings for the validation set
a,b = load_validation(set)

Img_Con_validation_paths = reshape_paths(a[0][0])
Aud_Con_validation_paths = reshape_paths(a[0][1])
Mix_Con_validation_paths = reshape_paths(a[0][2])

Img_Descon_validation_paths = reshape_paths(a[1][0])
Aud_Descon_validation_paths = reshape_paths(a[1][1])
Mix_Descon_validation_paths = reshape_paths(a[1][2])


Img_Con_validation_embeddings = reshape_embed(b[0][0])
Aud_Con_validation_embeddings = reshape_embed(b[0][1])
Mix_Con_validation_embeddings = reshape_embed(b[0][2])

Img_Descon_validation_embeddings = reshape_embed(b[1][0])
Aud_Descon_validation_embeddings = reshape_embed(b[1][1])
Mix_Descon_validation_embeddings = reshape_embed(b[1][2])


# Paths and Embeddings for the test set
a,b = load_test(set)

Img_Con_test_paths = reshape_paths(a[0][0])
Aud_Con_test_paths = reshape_paths(a[0][1])
Mix_Con_test_paths = reshape_paths(a[0][2])

Img_Descon_test_paths = reshape_paths(a[1][0])
Aud_Descon_test_paths = reshape_paths(a[1][1])
Mix_Descon_test_paths = reshape_paths(a[1][2])

Img_Con_test_embeddings = reshape_embed(b[0][0])
Aud_Con_test_embeddings = reshape_embed(b[0][1])
Mix_Con_test_embeddings = reshape_embed(b[0][2])

Img_Descon_test_embeddings = reshape_embed(b[1][0])
Aud_Descon_test_embeddings = reshape_embed(b[1][1])
Mix_Descon_test_embeddings = reshape_embed(b[1][2])



##
Conocidos = len(Img_Con_train_embeddings)
Desconocidos = len(Img_Descon_train_embeddings)
muestras_img = len(Img_Con_train_embeddings[0])
muestras_aud = len(Aud_Con_train_embeddings[0])

img_test = len(Img_Con_test_embeddings[0])
aud_test = len(Aud_Con_test_embeddings[0])

columnas = a[0][0].columns

In [3]:
print ('Known: ', Conocidos)
print ('Unknown: ', Desconocidos)
print ('Images per person (train): ', muestras_img)
print ('Audios per person (train): ', muestras_aud)
print ('Images per person (test): ', img_test)
print ('Audios per person (test): ', aud_test)


Conocidos:  46
Desconocidos:  46
Muestras por imagen:  40
Muestras por audio:  2
Muestras por imagen test:  30
Muestras por audio test:  1


### Start of the experiment

In [4]:
def ensayo (dataset_con,dataset_descon,modelo,ruido = -1,inliers = 1,title = ''):
    
    T_P = 0
    T_N = 0
    F_P = 0
    F_N = 0
    
    for i in range(len(dataset_con)):
        x = dataset_con[i]
        x = x.reshape(1,-1)
        y = modelo.predict(x)
        if y == inliers:
            T_P += 1
        else:
            F_N += 1
            
            
    for i in range(len(dataset_descon)):
        x = dataset_descon[i]
        x = x.reshape(1,-1)
        y = modelo.predict(x)
        if y == ruido:
            T_N += 1
        else:
            F_P += 1
            
    #plot_confussion(T_P,T_N,F_P,F_N,title)
    return T_P,T_N,F_P,F_N
    

In [4]:
Data_train = Mix_Con_train_embeddings.reshape(Conocidos*muestras_img,1024)
Data_test_con = Mix_Con_test_embeddings.reshape(Conocidos*img_test,1024)
Data_val_con = Mix_Con_validation_embeddings.reshape(Conocidos*img_test,1024)

Data_test_descon = Mix_Descon_test_embeddings.reshape(Desconocidos*img_test,1024)
Data_val_descon = Mix_Descon_validation_embeddings.reshape(Desconocidos*img_test,1024)

y = np.ones(len(Data_train))

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score,make_scorer


f1score = make_scorer(f1_score, average='macro')

resultados = pd.DataFrame(columns=['Modelo','Precision','Recall','F1','Acurracy'])

In [6]:
from sklearn.svm import OneClassSVM
# One Class SVM



parameters = {'kernel':('linear', 'rbf','poly'),'degree':(1,2,3,4),
              'nu':[0.01,0.05,0.1,0.5,0.9],'gamma':('scale','auto')}

OCSVM = OneClassSVM()
clf = GridSearchCV(OCSVM, parameters,scoring=f1score)
clf.fit(Data_train,y)

results = pd.DataFrame(clf.cv_results_)
B_OCSVM = clf.best_estimator_
clf.best_params_

# Best params Vox: {'degree': 1, 'gamma': 'scale', 'kernel': 'poly', 'nu': 0.01}
# Best params Ave  {'degree': 1, 'gamma': 'scale', 'kernel': 'linear', 'nu': 0.01}
# Best params Mobio {'degree': 1, 'gamma': 'scale', 'kernel': 'linear', 'nu': 0.01}

{'degree': 1, 'gamma': 'scale', 'kernel': 'linear', 'nu': 0.01}

In [7]:
Precision,Recall,F1,Acurracy = calculate_metrics(*ensayo(Data_test_con,Data_test_descon,B_OCSVM,-1,1,'Resultados de OCSVM con VoxCeleb'))
print ('Acurracy: ',Acurracy)
resultados.loc[0] = ['OCSVM',Precision,Recall,F1,Acurracy]

Acurracy:  0.7434782608695653


In [8]:
from sklearn.neighbors import LocalOutlierFactor
# Local Outlier Factor

parameters = {'n_neighbors':(2,5,10,15),'algorithm':('auto', 'ball_tree', 'kd_tree'),
                 'contamination':[0.01,0.05,0.1]}
clf = GridSearchCV(LocalOutlierFactor(novelty=True), parameters,scoring=f1score)
clf.fit(Data_train,y)

results = pd.DataFrame(clf.cv_results_)
B_LOF = clf.best_estimator_
clf.best_params_    

# Best params Vox: {'algorithm': 'auto', 'contamination': 0.01, 'n_neighbors': 2}
# Best params Ave {'algorithm': 'auto', 'contamination': 0.01, 'n_neighbors': 2}
# Best params Mobio {'algorithm': 'auto', 'contamination': 0.01, 'n_neighbors': 5}


{'algorithm': 'auto', 'contamination': 0.01, 'n_neighbors': 5}

In [9]:
Precision,Recall,F1,Acurracy = calculate_metrics(*ensayo(Data_test_con,Data_test_descon,B_LOF))
print ('Acurracy: ',Acurracy)
resultados.loc[1] = ['LOF',Precision,Recall,F1,Acurracy]


Acurracy:  0.6289855072463768


In [10]:
from sklearn.ensemble import IsolationForest
# Isolation Forest

parameters = {'n_estimators':(50,100,200,400,600,800),'contamination':(0.01,0.05,0.1,),
              'max_features':(1.0,0.9,0.8)}

clf = GridSearchCV(IsolationForest(), parameters,scoring=f1score)
clf.fit(Data_train,y)

results = pd.DataFrame(clf.cv_results_)
B_IF = clf.best_estimator_
clf.best_params_

# Best params Vox: {'contamination': 0.01, 'max_features': 0.9, 'n_estimators': 50}
# Best params Ave: {'contamination': 0.01, 'max_features': 0.8, 'n_estimators': 50}
# Best params Mobio: {'contamination': 0.01, 'max_features': 0.9, 'n_estimators': 100}

{'contamination': 0.01, 'max_features': 0.9, 'n_estimators': 100}

In [11]:
Precision,Recall,F1,Acurracy = calculate_metrics(*ensayo(Data_test_con,Data_test_descon,B_IF,-1,1,'Resultados de IF con AveRobot'))
print ('Acurracy: ',Acurracy)
resultados.loc[2] = ['IF',Precision,Recall,F1,Acurracy]

Acurracy:  0.5681159420289855


In [12]:
from sklearn.linear_model import SGDOneClassSVM

# SGD One Class SVM

parameters = {'nu':(0.05,0.01,0.1,0.2)}

clf = GridSearchCV(SGDOneClassSVM(), parameters,scoring=f1score)
clf.fit(Data_train,y)

results = pd.DataFrame(clf.cv_results_)
B_SGD = clf.best_estimator_
clf.best_params_

# Best params Vox: {'nu': 0.01}
# Best params Ave: {'nu': 0.01}
# Best params Mobio: {'nu': 0.01}

{'nu': 0.01}

In [13]:
Precision,Recall,F1,Acurracy = calculate_metrics(*ensayo(Data_test_con,Data_test_descon,B_SGD))
print ('Acurracy: ',Acurracy)
resultados.loc[3] = ['SGD',Precision,Recall,F1,Acurracy]

Acurracy:  0.7384057971014493


In [14]:
# DeepSVDD

from pyod.models.deep_svdd import DeepSVDD



# Aqui los outliers son 1 y los inliers son 0
y = np.zeros(len(Data_train))
svdd = DeepSVDD(verbose=0)
parameters = {'dropout_rate':(0.1,0.2,0.15),'contamination':(0.05,0.1,0.01),'hidden_neurons':((1024,512,256),(1024,512,256,128),(1024,512,256,128,64))}
clf = GridSearchCV(svdd, parameters,scoring=f1score)
clf.fit(Data_train,y)
results = pd.DataFrame(clf.cv_results_)
B_svdd = clf.best_estimator_
clf.best_params_

# Best params Vox: {'contamination': 0.01,'dropout_rate': 0.1,'hidden_neurons': (1024, 512, 256, 128, 64)}
# Best params Ave: {'contamination': 0.01,'dropout_rate': 0.2,'hidden_neurons': (1024, 512, 256, 128, 64)}
# Best paramsa Mobio: {'contamination': 0.01, 'dropout_rate': 0.2, 'hidden_neurons': (1024, 512, 256, 128, 64)}


2023-07-05 17:23:05.048428: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-05 17:23:05.151129: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-05 17:23:05.168040: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-05 17:23:05.516687: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

46/46 [==============================] - 2s 668us/step


2023-07-05 17:23:07.984608: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


46/46 [==============================] - 0s 517us/step


/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(


...

/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(


58/58 [==============================] - 0s 1ms/step


/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(


58/58 [==============================] - 0s 596us/step


{'contamination': 0.01,
 'dropout_rate': 0.2,
 'hidden_neurons': (1024, 512, 256, 128, 64)}

In [15]:
from pyod.models.deep_svdd import DeepSVDD
y = np.zeros(len(Data_train))
B_svdd = DeepSVDD(contamination=0.01,dropout_rate=0.2,hidden_neurons=(1024, 512, 256, 128, 64),verbose=0)
B_svdd.fit(Data_train,y)

Precision,Recall,F1,Acurracy = calculate_metrics(*ensayo(Data_test_con,Data_test_descon,B_svdd,1,0))
print ('Acurracy: ',Acurracy)
resultados.loc[4] = ['DeepSVDD',Precision,Recall,F1,Acurracy]

58/58 [==============================] - 0s 543us/step


/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(


1/1 [==============================] - 0s 10ms/step
Acurracy:  0.5539855072463769


In [16]:
resultados

,Modelo,Precision,Recall,F1,Acurracy
0,OCSVM,0.680451,0.918116,0.781616,0.743478
1,LOF,0.997207,0.258696,0.410817,0.628986
2,IF,0.539363,0.933333,0.683652,0.568116
3,SGD,0.678030,0.907971,0.776332,0.738406
4,DeepSVDD,0.534253,0.842029,0.653727,0.553986


In [17]:
from pyod.models.iforest import IForest

# IForest

parameters = {'n_estimators':(50,100,200,300,400),'contamination':(0.01,0.05,0.1),'max_features':(1,0.9,0.8)}

clf = GridSearchCV(IForest(), parameters,scoring=f1score)
clf.fit(Data_train,y)
results = pd.DataFrame(clf.cv_results_)
B_IF = clf.best_estimator_
clf.best_params_

# Best params Vox: {'contamination': 0.01, 'max_features': 0.8, 'n_estimators': 50}
# Best params Ave: {'contamination': 0.01, 'max_features': 0.9, 'n_estimators': 100}
# Best params MObio: {'contamination': 0.01, 'max_features': 0.8, 'n_estimators': 400}

/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised l

{'contamination': 0.01, 'max_features': 0.8, 'n_estimators': 400}

In [18]:
Precision,Recall,F1,Acurracy = calculate_metrics(*ensayo(Data_test_con,Data_test_descon,B_IF,1,0,'Resultados de IF con AveRobot'))
print ('Acurracy: ',Acurracy)   
resultados.loc[5] = ['IForest',Precision,Recall,F1,Acurracy]

Acurracy:  0.6735507246376812


In [19]:
from pyod.models.knn import KNN

# KNN

parameters = {'n_neighbors':(2,5,8,10,15),'contamination':(0.01,0.05,0.1),'method':('largest','mean','median')}

clf = GridSearchCV(KNN(), parameters,scoring=f1score)
clf.fit(Data_train,y)
results = pd.DataFrame(clf.cv_results_)
B_KNN = clf.best_estimator_
clf.best_params_
# Best params Vox: {'contamination': 0.01, 'method': 'largest', 'n_neighbors': 2}
# Best params Ave: {'contamination': 0.01, 'method': 'largest', 'n_neighbors': 2}
# Best params Mobio: {'contamination': 0.01, 'method': 'mean', 'n_neighbors': 2}

/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised l

{'contamination': 0.01, 'method': 'mean', 'n_neighbors': 2}

In [20]:
Precision,Recall,F1,Acurracy = calculate_metrics(*ensayo(Data_test_con,Data_test_descon,B_KNN,1,0))
print ('Acurracy: ',Acurracy)
resultados.loc[6] = ['KNN',Precision,Recall,F1,Acurracy]

Acurracy:  0.5659420289855073


In [21]:
from pyod.models.lof import LOF

# LOF

parameters = {'n_neighbors':(2,5,8,10,15),'contamination':(0.05,0.01,0.1)}

clf = GridSearchCV(LOF(), parameters,scoring=f1score)
clf.fit(Data_train,y)
results = pd.DataFrame(clf.cv_results_)
B_LOF = clf.best_estimator_
clf.best_params_

# Best params Vox: {'contamination': 0.05, 'n_neighbors': 2}
# Best params Ave: {'contamination': 0.01, 'n_neighbors': 2}
# Best params Mobio: {'contamination': 0.01, 'n_neighbors': 5}

/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised learning.
  warnings.warn(
/home/jose/miniconda3/envs/probando/lib/python3.9/site-packages/pyod/models/base.py:431: UserWarning: y should not be presented in unsupervised l

{'contamination': 0.01, 'n_neighbors': 5}

In [22]:
Precision,Recall,F1,Acurracy = calculate_metrics(*ensayo(Data_test_con,Data_test_descon,B_LOF,1,0))
print ('Acurracy: ',Acurracy)
resultados.loc[7] = ['LOF',Precision,Recall,F1,Acurracy]
    

Acurracy:  0.6289855072463768


In [23]:
resultados.to_csv('Resultados/resultados_'+set+'_Pyod.csv')

In [25]:
resultados

,Modelo,Precision,Recall,F1,Acurracy
0,OCSVM,0.680451,0.918116,0.781616,0.743478
1,LOF,0.997207,0.258696,0.410817,0.628986
2,IF,0.539363,0.933333,0.683652,0.568116
3,SGD,0.678030,0.907971,0.776332,0.738406
4,DeepSVDD,0.534253,0.842029,0.653727,0.553986
5,IForest,0.609411,0.966667,0.747548,0.673551
6,KNN,1.000000,0.131884,0.233035,0.565942
7,LOF,0.997207,0.258696,0.410817,0.628986
